# **Modeling the problem as a Relaxed Integer Problem** 


## **Relaxed integer problem** 

The following code is the same used for the integer problem with the exception that we have relaxed the binary variable  $y_j$ to allow it to take non-negative real numbers, so, it is not a binary variable anymore.

In [281]:
def hospital_relaxed_integer_model(n, p, w, h1, h2, h3_1_2, h3_3_4, h3_5_6, h3_7_8, h3_9_10, h4):
    """
    Implements and solves the hospital integer (binary) model.

    Parameters (inputs)
    ----------
    n: number of nurse-doctor tandems in a given day.
    p: number of patients tandems in a given day.
    w: 1-D array with the severity/importance of the patients.
    h1: maximum time seeing patients per nurse-doctor tandem in a given day.
    h2: maximum time spent by nurse-doctor tandem i seeing the patient j, for all i,j.
    h3_1_2: minimum time time spent in patient j, if w[j] in [1,2].
    h3_3_4: minimum time time spent in patient j, if w[j] in [3,4].
    h3_5_6: minimum time time spent in patient j, if w[j] in [5,6].
    h3_7_8: minimum time time spent in patient j, if w[j] in [7,8].
    h3_9_10: minimum time time spent in patient j, if w[j] in [9,10].
    h4: maximum time time spent in patient j.

    Returns (outputs)
    -------
    model: the Pyomo model output.
    t_optimal: a dictionary with the optimal values for the t_ij decision variables.
    y_optimal: a dictionary with the optimal values for the y_j decision variables.
    f_optimal: the optimal value of the objective function.
    """

    # Defining h3 function.
    def h3(w):
        if w in [1, 2] :
           return h3_1_2
        if w in [3, 4] :
           return h3_3_4
        elif w in [5, 6] :
           return h3_5_6
        elif w in [7, 8] :
           return h3_7_8
        elif w in [9, 10] :
           return h3_9_10

    # Initialize a model.
    model = ConcreteModel()

    # Initialize ranges for tandems and patients.
    model.N = range(0,n)
    model.P = range(0,p)

    # Defining t_ij decision variables as non-negative reals (t_ij >= 0).
    model.t = Var(model.N, model.P, domain=NonNegativeReals) 

    # Defining y_j as as non-negative real (y_j >= 0).
    model.y = Var(model.P, domain=NonNegativeReals)

    # Objective function to maximize the weighted sum of time spent with patients
    model.cost = Objective(expr=sum(model.t[i,j] * w[j] for i in model.N for j in model.P), sense=maximize)

    # Constraints
    model.constraints = ConstraintList()

    # The total time spent by each tandem on all patients must be at most h1.
    for i in model.N:
        model.constraints.add(sum(model.t[i,j] for j in model.P) <= h1)

    # The time spent by tandem i on patient j must be at most h2.
    for i in model.N:
        for j in model.P:
            model.constraints.add(model.t[i, j] <= h2)
    
    # The total time a patient is seen by all tandems must be at least h3(w[j])
    for j in model.P:
        model.constraints.add(sum(model.t[i,j] for i in model.N) >= h3(w[j]))

    # The total time a patient is seen by all tandems must be at most h4.
    for j in model.P:
        model.constraints.add(sum(model.t[i,j] for i in model.N) <= h4)

    # y[j] is 1 if sum(model.t[i,j] for i in model.N) > 0.
    M = 1000000
    for j in model.P:
        model.constraints.add(sum(model.t[i,j] for i in model.N) <= model.y[j] * M)
         
    # y[j] is 0 if sum(model.t[i,j] for i in model.N) = 0
    epsilon = 0.000001
    for j in model.P:
        model.constraints.add(epsilon * model.y[j] <= sum(model.t[i,j] for i in model.N))

    # The number of patients seen (sum(model.y[j] for j in model.P)) must be equal to the number of patients (p).
    model.constraints.add(sum(model.y[j] for j in model.P) == p)

    # Load dual information
    # model.dual = Suffix(direction=Suffix.IMPORT)

    # Solve the model
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=False)  # Set tee=True to show solver output

    # Check solver status and termination condition
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        #print("Solution Found")
        # Extract optimal values for variables and objective
        t_optimal = {(i, j): model.t[i, j].value for i in model.N for j in model.P}
        y_optimal = {j: model.y[j].value for j in model.P}
        f_optimal = model.cost.expr()
    elif results.solver.termination_condition == TerminationCondition.infeasible:
        #print("No feasible solution")
        t_optimal, y_optimal, f_optimal = None, None, None
    else:
        #print("No feasible solution")
        t_optimal, y_optimal, f_optimal = None, None, None

    return model, t_optimal, y_optimal, f_optimal

## **Preliminary analysis**

In this section we are going to do a preliminary numerical analysis of the problem posed above, for a given set of parameters.

In [282]:
# Parameters definition:
n=50; p=500; h1=6; h2=1; h4=4

h3_1_2 = 0
h3_3_4 = 0.30 
h3_5_6 = 0.55  
h3_7_8 = 0.80 
h3_9_10 = 1 
    
np.random.seed(123)
w = np.random.randint(1, 11, p) 

# Problem execution:
model, t_optimal, y_optimal, f_optimal = hospital_relaxed_integer_model(n, p, w, h1, h2, h3_1_2, h3_3_4, h3_5_6, h3_7_8, h3_9_10, h4)

### **Optimal values**

 **Decision variables**

The optimal values for the $t_{ij}$ decision variables for the given set of parameters are the following:

In [283]:
t_optimal

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (0, 5): 0.0,
 (0, 6): 0.0,
 (0, 7): 0.0,
 (0, 8): 0.0,
 (0, 9): 0.0,
 (0, 10): 0.0,
 (0, 11): 0.0,
 (0, 12): 0.0,
 (0, 13): 0.0,
 (0, 14): 0.0,
 (0, 15): 0.0,
 (0, 16): 0.0,
 (0, 17): 0.0,
 (0, 18): 0.0,
 (0, 19): 0.0,
 (0, 20): 0.0,
 (0, 21): 0.0,
 (0, 22): 0.0,
 (0, 23): 0.0,
 (0, 24): 0.0,
 (0, 25): 0.0,
 (0, 26): 0.0,
 (0, 27): 0.0,
 (0, 28): 0.0,
 (0, 29): 0.0,
 (0, 30): 0.0,
 (0, 31): 0.0,
 (0, 32): 0.0,
 (0, 33): 0.0,
 (0, 34): 0.0,
 (0, 35): 0.0,
 (0, 36): 0.0,
 (0, 37): 0.0,
 (0, 38): 0.0,
 (0, 39): 0.0,
 (0, 40): 0.0,
 (0, 41): 0.0,
 (0, 42): 0.0,
 (0, 43): 0.0,
 (0, 44): 0.0,
 (0, 45): 0.0,
 (0, 46): 0.0,
 (0, 47): 0.0,
 (0, 48): 0.0,
 (0, 49): 0.0,
 (0, 50): 0.0,
 (0, 51): 0.0,
 (0, 52): 0.0,
 (0, 53): 0.0,
 (0, 54): 0.0,
 (0, 55): 0.0,
 (0, 56): 0.0,
 (0, 57): 0.0,
 (0, 58): 0.0,
 (0, 59): 0.0,
 (0, 60): 0.0,
 (0, 61): 0.0,
 (0, 62): 0.0,
 (0, 63): 0.0,
 (0, 64): 0.0,
 (0, 65): 0.0,
 (0, 66): 0.0,
 (0, 

The optimal values for the $y_{j}$ decision variables for the given set of parameters are the following:

In [284]:
y_optimal

{0: 3e-07,
 1: 3e-07,
 2: 8e-07,
 3: 0.0,
 4: 3e-07,
 5: 1e-06,
 6: 8e-07,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 1e-06,
 11: 0.0,
 12: 0.0,
 13: 4e-06,
 14: 3e-07,
 15: 5.5000000000002e-07,
 16: 0.0,
 17: 0.0,
 18: 5.5e-07,
 19: 0.0,
 20: 8e-07,
 21: 3e-07,
 22: 3e-07,
 23: 5.5e-07,
 24: 8e-07,
 25: 3e-07,
 26: 5.5e-07,
 27: 1e-06,
 28: 0.0,
 29: 8e-07,
 30: 4e-06,
 31: 3e-07,
 32: 5.5e-07,
 33: 8e-07,
 34: 0.0,
 35: 5.5e-07,
 36: 8e-07,
 37: 3e-07,
 38: 0.0,
 39: 1e-06,
 40: 3e-07,
 41: 5.5e-07,
 42: 0.0,
 43: 3e-07,
 44: 8e-07,
 45: 3e-07,
 46: 5.5e-07,
 47: 5.5e-07,
 48: 8e-07,
 49: 3e-07,
 50: 0.0,
 51: 8e-07,
 52: 5.5e-07,
 53: 8e-07,
 54: 8e-07,
 55: 8e-07,
 56: 0.0,
 57: 5.5e-07,
 58: 8e-07,
 59: 4e-06,
 60: 3e-07,
 61: 5.5e-07,
 62: 1e-06,
 63: 0.0,
 64: 3e-07,
 65: 0.0,
 66: 0.0,
 67: 3e-07,
 68: 5.5e-07,
 69: 4e-06,
 70: 0.0,
 71: 1e-06,
 72: 0.0,
 73: 8e-07,
 74: 3e-07,
 75: 3e-07,
 76: 5.5e-07,
 77: 4e-06,
 78: 7.99999999999997e-07,
 79: 4.00000000000001e-06,
 80: 3e-07,
 81: 3e-

For example, the time spent in patient $j=5$ by tandem $i=1$, i.e. $t_{15}$, is:

In [285]:
t_optimal[0,4] # 0 h

0.0

**Objective function**

In this case the optimal value for the objective function is the following:

In [286]:
f_optimal

2316.600000000006

### **Time spent in patients**

Now we are going to analyze the time spent in the patients, in this particular case.

We built a function that computes the time spent seeing patient j in a given day .
It's important to notice that a patient could be seen by several nurse-doctor tandems.

In [287]:
# Time spent seeing patient j in a given day (he could be seen by several nurse-doctor tandems).
def time_seeing_patient(j):
    return np.sum([t_optimal[i, j] for i in range(0,n)])

For example, total time spent in patient $j=45$, that is, $\sum_{i=1}^n t_{i45}$, is the following:

In [288]:
time_seeing_patient(44)

0.8

Using the previous function we can collect the total time spent in each patient in an array.

In [289]:
total_times_per_patient = np.array([time_seeing_patient(j) for j in range(0,p)])

In [290]:
total_times_per_patient

array([0.3 , 0.3 , 0.8 , 0.  , 0.3 , 1.  , 0.8 , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 4.  , 0.3 , 0.55, 0.  , 0.  , 0.55, 0.  , 0.8 , 0.3 ,
       0.3 , 0.55, 0.8 , 0.3 , 0.55, 1.  , 0.  , 0.8 , 4.  , 0.3 , 0.55,
       0.8 , 0.  , 0.55, 0.8 , 0.3 , 0.  , 1.  , 0.3 , 0.55, 0.  , 0.3 ,
       0.8 , 0.3 , 0.55, 0.55, 0.8 , 0.3 , 0.  , 0.8 , 0.55, 0.8 , 0.8 ,
       0.8 , 0.  , 0.55, 0.8 , 4.  , 0.3 , 0.55, 1.  , 0.  , 0.3 , 0.  ,
       0.  , 0.3 , 0.55, 4.  , 0.  , 1.  , 0.  , 0.8 , 0.3 , 0.3 , 0.55,
       4.  , 0.8 , 4.  , 0.3 , 0.3 , 0.3 , 0.3 , 1.  , 0.8 , 4.  , 0.8 ,
       0.8 , 0.3 , 4.  , 0.8 , 0.8 , 0.8 , 0.  , 0.3 , 0.55, 0.3 , 0.  ,
       0.  , 0.55, 1.  , 0.8 , 1.  , 1.  , 0.  , 0.  , 0.3 , 0.  , 0.3 ,
       0.55, 0.8 , 0.8 , 0.  , 0.55, 0.3 , 0.3 , 0.8 , 0.8 , 1.  , 0.8 ,
       0.55, 0.55, 0.8 , 0.  , 0.  , 1.  , 1.  , 1.  , 0.55, 1.  , 0.8 ,
       0.  , 0.8 , 1.  , 0.8 , 1.  , 0.  , 0.8 , 0.  , 0.8 , 1.  , 1.  ,
       0.8 , 0.  , 0.3 , 0.  , 1.  , 0.8 , 0.55, 0.

### **Average time spent per patient**

Now we can compute some statistics over the time spent in patients, like for example the average time spent per patient.

In [291]:
np.mean(total_times_per_patient)

0.6000000000000005

### **Max time spent per patient**

The maximum time spent in a single patient is enforced by $h_4$ constriant, that in this case has been fixed to $4$.

In [292]:
np.max(total_times_per_patient) # 4 hours

4.000000000000008

### **Min time spent per patient**

The minimun time spent in a single patient is set by $h_3^{12}$, in this case, that has been fixed to $0.1$.

In [293]:
np.min(total_times_per_patient) 

0.0

### **Median of time spent per patient**

In [294]:
np.median(total_times_per_patient) 

0.55

### **$75$-quantile of time spent per patient**

In [295]:
np.quantile(total_times_per_patient, 0.75)

0.8

### **$25$-quantile time spent per patient**

In [296]:
np.quantile(total_times_per_patient, 0.25)

0.3

### **Number of patients seen**

The number of patients seen is $p=500$ even though we have set $h_3^{12}=0$, so the new constraint is working well. In addition, we can see as the number of patient seen match with $\sum_{j=1}^p y_j$, which is something that must always be fulfilled in this new version of the problem.

In [297]:
np.sum(total_times_per_patient > 0)

396

In [298]:
np.sum(list(y_optimal.values()))

500.0

## **Comparison with the integer (not relaxed) model**

If the binary variable $y_j$ in the integer model is relaxed in the interval $[0,1]\subset \mathbb{R}$, no changes are produced, since $y_j$ carries on working as a binary variable, so it is still performing like it did in the non relaxed integer model.

However, if $y_j$ is relaxed to $\mathbb{R}^+$ (positive real numbers), then the results of the relaxed model are slightly different to those of the non relaxed, concretely the number of patient seen $(396)$ is not maximum $(p=500)$, so the particular new constraint of this model (with respect to the linear one) is not working, since it is not enforcing the number of patients seen to be equal to $p$, and this is because $y_j$ is not working as a binary variable, and, therefore, it is not longer useful to impose that condition.